## Загружаем модель, датасеты и семплируем таргеты

In [1]:
# для импорта из родительского каталога 
import os
import sys
import inspect

#currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
#parentdir = os.path.dirname(currentdir)
#sys.path.insert(0, parentdir) 

In [2]:
os.getcwd()

'C:\\Users\\filip\\PycharmProjects\\Diplom\\rich-gan-tf-2021_work\\baseline_metrics'

In [3]:
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

In [4]:
print(currentdir)
print(parentdir)

C:\Users\filip\PycharmProjects\Diplom\rich-gan-tf-2021_work\baseline_metrics
C:\Users\filip\PycharmProjects\Diplom\rich-gan-tf-2021_work


In [5]:
print(sys.path)

['C:\\Users\\filip\\PycharmProjects\\Diplom\\rich-gan-tf-2021_work', 'C:\\Users\\filip\\PycharmProjects\\Diplom\\rich-gan-tf-2021_work\\baseline_metrics', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN\\python310.zip', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN\\DLLs', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN\\lib', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN', '', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN\\lib\\site-packages', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN\\lib\\site-packages\\win32', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN\\lib\\site-packages\\win32\\lib', 'C:\\Users\\filip\\anaconda3\\envs\\RICH_GAN\\lib\\site-packages\\Pythonwin']


In [6]:
import pandas as pd
from load_model import MetricsModel

In [7]:
os.system('python ./richgan/utils/test_file.py')

2

In [8]:
os.getcwd()

'C:\\Users\\filip\\PycharmProjects\\Diplom\\rich-gan-tf-2021_work\\baseline_metrics'

In [9]:
config_file= r'C:\Users\filip\PycharmProjects\Diplom\rich-gan-tf-2021_work\saved_models\ModelMuonNormLayerMC\config.yaml'
chpt_dir =r'C:\Users\filip\PycharmProjects\Diplom\rich-gan-tf-2021_work\saved_models'

In [10]:
config_file

'C:\\Users\\filip\\PycharmProjects\\Diplom\\rich-gan-tf-2021_work\\saved_models\\ModelMuonNormLayerMC\\config.yaml'

In [11]:
config_file = r'C:\Users\filip\PycharmProjects\Diplom\rich-gan-tf-2021_work\saved_models\ModelMuonNormLayerMC\config.yaml'
chpt_dir = r'C:\Users\filip\PycharmProjects\Diplom\rich-gan-tf-2021_work\saved_models\ModelMuonNormLayerMC'
m_model = MetricsModel(config_file, chpt_dir)
targets_data_dict = m_model.get_targets()

AssertionError: 

In [ ]:
targets_data_dict.keys()

### Признаки

In [ ]:
# при обучении используется только: 
# Brunel_P, Brunel_ETA, nSPDhits
targets_data_dict['features']

## Рисуем промежуточные графики 

In [ ]:
%matplotlib inline

from plot_targets import plot_Hist1D

In [ ]:
dll_list = plot_Hist1D(targets_data_dict['features'],
                targets_data_dict['targets_real'],
                targets_data_dict['targets_fake_gan'],
                targets_data_dict['targets_fake_kde'],
                targets_data_dict['weights'],
                model_name='GAN (no wght)', model_name2='KDE')

### Эффектировности удобные для просмотра 

In [ ]:
from plot_efficiency import plot_EfficiencyDouble

In [ ]:
# чтобы удобнее смотреть, нужно отобрать только немного эффективностей
thresholds = [[0.95], [0.05], [0.95], [0.05]]

In [ ]:
efficiency_dict = plot_EfficiencyDouble(targets_data_dict['features'],
                                         targets_data_dict['targets_real'],
                                         targets_data_dict['targets_fake_gan'],
                                         targets_data_dict['targets_fake_kde'],
                                         targets_data_dict['weights'],
                                         thresholds = thresholds,
                                         model_names=['GAN  (no wght)', 'KDE'])

### Эффективности нужные для итоговых метрик 

In [ ]:
# нужно взять все эффетивности 
thresholds = [[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95],
            [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95],
            [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95],
            [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]]

In [ ]:
efficiency_dict = plot_EfficiencyDouble(targets_data_dict['features'],
                                         targets_data_dict['targets_real'],
                                         targets_data_dict['targets_fake_gan'],
                                         targets_data_dict['targets_fake_kde'],
                                         targets_data_dict['weights'],
                                         thresholds = thresholds,
                                         model_names=['GAN  (no wght)', 'KDE'])

In [ ]:
efficiency_dict['EfficiencyMaker:1']['GAN']['eff_ratio_RichDLLe_vs_Brunel_P_at_[0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]']

## Итоговые метрики

In [ ]:
from math import sqrt

In [ ]:
# вычисляем расстояние по идее Артема
def calculate_dist(bins):
    dis_sum = 0
    for bin_x in bins.iterrows():
        eff_ratio = bin_x[1][0]
        err_low = bin_x[1][1]
        err_high = bin_x[1][2]
        if eff_ratio <= 1:
            dist = ((1-eff_ratio)/err_high)**2
        else:
            dist = ((1-eff_ratio)/err_low)**2
        dis_sum += dist
    return dis_sum

In [ ]:
# если брать корень в самом конце
def dis_for_model(raw_output_dict, model_name, percents):
    dis_sum = 0
    for eff_name in list(raw_output_dict['EfficiencyMaker:1'][model_name]):
        # чтобы не было слишком сильной корреляции
        if '_reverse' in eff_name or '_P_T_' in eff_name:
            continue
        all_percent_dist = 0
        for percent in percents:
            eff_ratio_column = 'eff_ratio_' + str(percent)
            eff_ratio_err_low_column = 'eff_ratio_err_low_' + str(percent)
            eff_ratio_err_high_column = 'eff_ratio_err_high_' + str(percent)
            bins = raw_output_dict['EfficiencyMaker:1'][model_name][eff_name][[eff_ratio_column,
                                                                               eff_ratio_err_low_column,
                                                                               eff_ratio_err_high_column]]
            percent_dist = calculate_dist(bins)
            # складываем для всех процентов
            all_percent_dist += percent_dist

        dis_sum += all_percent_dist
    return sqrt(dis_sum)

In [ ]:
# Если брать корень после каждого графика
def dis_for_model_end(raw_output_dict, model_name, percents):
    dis_sum = 0
    for eff_name in list(raw_output_dict['EfficiencyMaker:1'][model_name]):
        # чтобы не было слишком сильной корреляции
        if '_reverse' in eff_name or '_P_T_' in eff_name:
            continue
        all_percent_dist = 0
        for percent in percents:
            eff_ratio_column = 'eff_ratio_'+str(percent)
            eff_ratio_err_low_column = 'eff_ratio_err_low_'+str(percent)
            eff_ratio_err_high_column = 'eff_ratio_err_high_'+str(percent)
            bins = raw_output_dict['EfficiencyMaker:1'][model_name][eff_name][[eff_ratio_column,
                                                                               eff_ratio_err_low_column,
                                                                               eff_ratio_err_high_column]]
            percent_dist = calculate_dist(bins)
            # складываем для всех процентов
            all_percent_dist += percent_dist

        dis_sum += sqrt(all_percent_dist)
    return dis_sum


In [ ]:
def compare_metrics(efficiency_dict):
    percents_list = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]
    dist_list = {'GAN': [], 'KDE': []}
    for percent in percents_list:
        dis_sum = dis_for_model(raw_output_dict = efficiency_dict, model_name='GAN', percents=[percent])
        dist_list['GAN'].append(dis_sum)
        dis_sum = dis_for_model(raw_output_dict = efficiency_dict, model_name='KDE', percents=[percent])
        dist_list['KDE'].append(dis_sum)

    print('Берем корень в самом конце:')
    print('GAN:', dist_list['GAN'][-1])
    print('KDE:', dist_list['KDE'][-1])

    dist_list2 = {'GAN': [], 'KDE': []}
    for percent in percents_list:
        dis_sum = dis_for_model_end(raw_output_dict = efficiency_dict, model_name='GAN', percents=[percent])
        dist_list2['GAN'].append(dis_sum)
        dis_sum = dis_for_model_end(raw_output_dict = efficiency_dict, model_name='KDE', percents=[percent])
        dist_list2['KDE'].append(dis_sum)

    print('Берем корень в после каждого графика:')
    print('GAN:', dist_list2['GAN'][-1])
    print('KDE:', dist_list2['KDE'][-1])
    return  dist_list, dist_list2

In [ ]:
dist_list, dist_list2 = compare_metrics(efficiency_dict)